In [2]:
from dino_finder import DinoFinder
import os 
import cv2
from PIL import Image
import numpy as np

In [3]:
CONFIG_PATH = "GroundingDINO/groundingdino/config/GroundingDINO_SwinT_OGC.py"
WEIGHTS_PATH = os.path.join("weights", "groundingdino_swint_ogc.pth")
SAM_WEIGHTS = "weights/sam_vit_h_4b8939.pth"

In [5]:
#utils 
def real_coords(x, img):
    y = x.clone() if isinstance(x, torch.Tensor) else np.copy(x)
    w = img.shape[1]
    h = img.shape[0]
    y[0] = w * (x[0] - x[2] / 2)  # top left x
    y[1] = h * (x[1] - x[3] / 2)  # top left y
    y[2] = w * (x[0] + x[2] / 2)  # bottom right x
    y[3] = h * (x[1] + x[3] / 2)  # bottom right y
    return y


def create_mask_collage(box_arr, cls_arr, img):
    collages = []
    imgc = []
    col = Image.new("RGBA", (img.shape[1], img.shape[0]))
    for box in box_arr:
        pil_img, coord, _crop_img = crop_image(box, img)
        col.paste(pil_img, coord)
    return col

def show_mask(mask,  random_color=True):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    return mask_image

def crop_image(box, img):
    rc = real_coords(np.array(box), img).tolist()
    rc = [int(x) for x in rc]
    crop_img = img[rc[1] : rc[3], rc[0] : rc[2]]
    pil_img = Image.fromarray(crop_img)
    return pil_img, (rc[0],rc[1]),crop_img

def filter_result_by_prompt(prompt, img):
    binary_img = None
    all_binary_mask = np.zeros([img.shape[0], img.shape[1]], dtype=np.uint8) 
    for idx, box in enumerate(finder.boxes):
        if finder.cls[idx] == prompt:
            pil_img, coord, crop_img = crop_image(np.array(box), img) 
            sam_res, scores, logit = masker.sam_pred_from_img(img, np.array([int(x) for x in real_coords(np.array(box), img)]))
            detections = sv.Detections(
                xyxy=sv.mask_to_xyxy(masks=sam_res),
                mask=sam_res
            )
            detections = detections[detections.area == np.max(detections.area)]
            anno_img = mask_annotator.annotate(scene=img.copy(), detections=detections)
            highest_score_idx = int(np.where(scores == np.max(scores))[0])
            binary_img = np.array(sam_res[highest_score_idx], dtype=np.uint8)
            binary_img[binary_img > 0] = 255
            all_binary_mask += binary_img        
    masked_img = cv2.bitwise_and(img, img, mask=all_binary_mask) 
    return masked_img, all_binary_mask 

def find_from_prompt(image_path, prompt, confidence, text_confidence):
    print(f"PROOMPT : {prompt, type(prompt)}")
    boxes, conf, cls, an = finder.find_by_prompt(
        prompt=prompt,
        bt=(confidence / 100),
        tt=(text_confidence / 100),
        image_path=image_path,
    )
    an = cv2.cvtColor(an, cv2.COLOR_BGR2RGB)
    collage = create_mask_collage(boxes, cls, image_path)
    return an, collage


def find_from_image(image_pth, prompt, bt, tt):
    text_prompt = prompt
    bt = bt
    tt = tt
    imgsrc, img = load_image(image_pth)
    boxes, logits, phrases = predict(
        model=model, image=img, caption=text_prompt, box_threshold=bt, text_threshold=tt
    )
    annotated_image = annotate(imgsrc, boxes, logits, phrases)
    cv2.imwrite(f"results/{os.path.basename(image_pth)}.png", annotated_image)
    return boxes.tolist()


In [7]:
from groundingdino.util.inference import load_model, load_image, predict, annotate
model = load_model(CONFIG_PATH, WEIGHTS_PATH)
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
img = "/home/hb/Pictures/test_female.png"
result = find_from_image(image_pth=img, prompt="human head", bt =0.5, tt =0.22)
print(result)

/home/hb/auto_label_GroundDINO/env/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


final text_encoder_type: bert-base-uncased


/home/hb/auto_label_GroundDINO/env/lib/python3.11/site-packages/transformers/modeling_utils.py:905: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
/home/hb/auto_label_GroundDINO/env/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/hb/auto_label_GroundDINO/env/lib/python3.11/site-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


[[0.5038403868675232, 0.4861435890197754, 0.44026246666908264, 0.6965095400810242]]
